# Chapter 24: Deployment Strategies

Deploying a Next.js application involves more than just pushing code to a repository. Modern deployment strategies must account for serverless architecture, edge computing, containerization, and continuous integration pipelines. Whether you're deploying to Vercel's optimized platform, self-hosting on Docker, or generating static exports for CDN distribution, understanding these deployment patterns ensures your application scales reliably and performs optimally in production.

By the end of this chapter, you'll master zero-configuration Vercel deployments, self-hosting with Node.js and Docker, static export generation for edge networks, configuring CI/CD pipelines with GitHub Actions, managing environment-specific configurations, and implementing production monitoring and analytics.

## 24.1 Vercel Deployment

Vercel provides the optimal platform for Next.js with zero-config deployments and automatic optimizations.

### Zero-Configuration Deployment

Deploy to Vercel with minimal setup:

```bash
# Install Vercel CLI
npm i -g vercel

# Login to Vercel
vercel login

# Deploy from project root
vercel

# Deploy to production
vercel --prod

# Link existing project
vercel link
```

```json
// package.json
{
  "scripts": {
    "dev": "next dev",
    "build": "next build",
    "start": "next start",
    "vercel-build": "prisma generate && next build"
  }
}
```

### Vercel-Specific Configuration

Optimize builds with `vercel.json`:

```json
// vercel.json
{
  "version": 2,
  "builds": [
    {
      "src": "package.json",
      "use": "@vercel/next"
    }
  ],
  "routes": [
    {
      "src": "/api/webhook",
      "methods": ["POST"],
      "dest": "/api/webhook"
    }
  ],
  "headers": [
    {
      "source": "/api/(.*)",
      "headers": [
        {
          "key": "Cache-Control",
          "value": "no-cache, no-store, must-revalidate"
        }
      ]
    },
    {
      "source": "/_next/static/(.*)",
      "headers": [
        {
          "key": "Cache-Control",
          "value": "public, max-age=31536000, immutable"
        }
      ]
    }
  ],
  "crons": [
    {
      "path": "/api/cron/daily-cleanup",
      "schedule": "0 0 * * *"
    }
  ],
  "functions": {
    "app/api/**/*.ts": {
      "maxDuration": 30
    }
  }
}
```

### Environment Variables on Vercel

Manage environment-specific settings:

```bash
# Add environment variables via CLI
vercel env add DATABASE_URL
vercel env add NEXTAUTH_SECRET
vercel env add STRIPE_SECRET_KEY production

# Pull environment variables locally
vercel env pull .env.local

# List all environment variables
vercel env ls
```

```typescript
// lib/vercel.ts
// Detect Vercel environment
export const isVercel = process.env.VERCEL === '1';
export const vercelEnv = process.env.VERCEL_ENV; // 'production' | 'preview' | 'development'
export const vercelUrl = process.env.VERCEL_URL; // Automatic deployment URL

// Dynamic base URL based on environment
export function getBaseUrl(): string {
  if (process.env.VERCEL_ENV === 'production') {
    return 'https://yourdomain.com';
  }
  if (process.env.VERCEL_URL) {
    return `https://${process.env.VERCEL_URL}`;
  }
  return 'http://localhost:3000';
}
```

## 24.2 Self-Hosting Options

Deploy Next.js on your own infrastructure for complete control.

### Node.js Server Deployment

Run Next.js as a traditional Node.js application:

```javascript
// next.config.js
module.exports = {
  output: 'standalone', // Creates optimized standalone build
  // Reduces image optimization load (if not using Next.js Image Optimization API)
  images: {
    unoptimized: true, // Or configure custom loader
  },
};
```

```dockerfile
# Dockerfile for Node.js deployment (see full Docker section below)
FROM node:18-alpine AS base

# Production deployment script
# server.js (automatically generated with output: 'standalone')
const { createServer } = require('http');
const { parse } = require('url');
const next = require('next');

const dev = process.env.NODE_ENV !== 'production';
const hostname = 'localhost';
const port = process.env.PORT || 3000;

const app = next({ dev, hostname, port });
const handle = app.getRequestHandler();

app.prepare().then(() => {
  createServer(async (req, res) => {
    try {
      const parsedUrl = parse(req.url, true);
      await handle(req, res, parsedUrl);
    } catch (err) {
      console.error('Error occurred handling', req.url, err);
      res.statusCode = 500;
      res.end('internal server error');
    }
  }).listen(port, (err) => {
    if (err) throw err;
    console.log(`> Ready on http://${hostname}:${port}`);
  });
});
```

### PM2 Process Management

Use PM2 for production process management:

```javascript
// ecosystem.config.js
module.exports = {
  apps: [
    {
      name: 'nextjs-app',
      script: './node_modules/next/dist/bin/next',
      args: 'start',
      instances: 'max', // Use all CPU cores
      exec_mode: 'cluster',
      env: {
        NODE_ENV: 'production',
        PORT: 3000,
      },
      env_production: {
        NODE_ENV: 'production',
        PORT: 3000,
      },
      error_file: './logs/err.log',
      out_file: './logs/out.log',
      log_date_format: 'YYYY-MM-DD HH:mm:ss Z',
      merge_logs: true,
      max_memory_restart: '1G',
      restart_delay: 3000,
      max_restarts: 5,
      min_uptime: '10s',
    },
  ],
};
```

```bash
# Start with PM2
pm2 start ecosystem.config.js --env production

# Save PM2 config to restart on boot
pm2 save
pm2 startup

# Monitoring
pm2 monit
pm2 logs
```

## 24.3 Docker Containerization

Containerize Next.js for consistent deployments across environments.

### Multi-Stage Production Dockerfile

Optimized Docker build for minimal image size:

```dockerfile
# Dockerfile
FROM node:18-alpine AS base

# Install dependencies only when needed
FROM base AS deps
RUN apk add --no-cache libc6-compat
WORKDIR /app

# Install dependencies based on the preferred package manager
COPY package.json yarn.lock* package-lock.json* pnpm-lock.yaml* ./
RUN \
  if [ -f yarn.lock ]; then yarn --frozen-lockfile; \
  elif [ -f package-lock.json ]; then npm ci; \
  elif [ -f pnpm-lock.yaml ]; then yarn global add pnpm && pnpm i --frozen-lockfile; \
  else echo "Lockfile not found." && exit 1; \
  fi

# Rebuild the source code only when needed
FROM base AS builder
WORKDIR /app
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Build arguments for environment variables
ARG DATABASE_URL
ENV DATABASE_URL=${DATABASE_URL}

# Build the application
RUN npm run build

# Production image, copy all the files and run next
FROM base AS runner
WORKDIR /app

ENV NODE_ENV production

RUN addgroup --system --gid 1001 nodejs
RUN adduser --system --uid 1001 nextjs

# Copy standalone output
COPY --from=builder /app/public ./public
COPY --from=builder --chown=nextjs:nodejs /app/.next/standalone ./
COPY --from=builder --chown=nextjs:nodejs /app/.next/static ./.next/static

USER nextjs

EXPOSE 3000

ENV PORT 3000
ENV HOSTNAME "0.0.0.0"

HEALTHCHECK --interval=30s --timeout=3s --start-period=40s \
  CMD curl -f http://localhost:3000/api/health || exit 1

CMD ["node", "server.js"]
```

```javascript
// next.config.js for Docker
module.exports = {
  output: 'standalone', // Required for Docker optimization
  // Enable if using Next.js Image Optimization with external loader
  images: {
    domains: ['localhost'],
  },
};
```

### Docker Compose Setup

Local development and production orchestration:

```yaml
# docker-compose.yml
version: '3.8'

services:
  app:
    build:
      context: .
      dockerfile: Dockerfile
      args:
        - DATABASE_URL=${DATABASE_URL}
    ports:
      - "3000:3000"
    environment:
      - DATABASE_URL=${DATABASE_URL}
      - NEXTAUTH_SECRET=${NEXTAUTH_SECRET}
      - REDIS_URL=redis://redis:6379
    depends_on:
      - redis
      - db
    networks:
      - app-network
    restart: unless-stopped

  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"
    volumes:
      - redis-data:/data
    networks:
      - app-network

  db:
    image: postgres:15-alpine
    environment:
      POSTGRES_USER: user
      POSTGRES_PASSWORD: password
      POSTGRES_DB: nextjs
    volumes:
      - postgres-data:/var/lib/postgresql/data
    ports:
      - "5432:5432"
    networks:
      - app-network

volumes:
  redis-data:
  postgres-data:

networks:
  app-network:
    driver: bridge
```

## 24.4 Static Exports

Generate static HTML for deployment to any static host or CDN.

### Static Export Configuration

Configure Next.js for static generation:

```javascript
// next.config.js
module.exports = {
  output: 'export',
  distDir: 'dist', // Output directory
  
  // Required for static export
  images: {
    unoptimized: true, // Static export requires unoptimized images
  },
  
  // Trailing slashes for static hosting compatibility
  trailingSlash: true,
  
  // Optional: Disable server-side features not supported in static export
  experimental: {
    // Some experimental features may not work with static export
  },
};
```

```bash
# Build static export
next build

# Deploy dist folder to any static host
# Netlify, Cloudflare Pages, AWS S3, etc.
```

### Handling Dynamic Routes in Static Export

Generate static pages for dynamic routes:

```typescript
// app/blog/[slug]/page.tsx
// For static export, you MUST generateStaticParams

export async function generateStaticParams() {
  // In real scenario, fetch from CMS or API
  const posts = await fetch('https://api.example.com/posts').then(r => r.json());
  
  return posts.map((post) => ({
    slug: post.slug,
  }));
}

// Or for catch-all routes
// app/docs/[...slug]/page.tsx
export async function generateStaticParams() {
  const docs = [
    { slug: ['getting-started'] },
    { slug: ['getting-started', 'installation'] },
    { slug: ['api', 'reference'] },
  ];
  
  return docs;
}
```

### Static Export with Client-Side Data

Fetch data client-side for dynamic content in static sites:

```typescript
// app/dashboard/page.tsx
'use client';

import useSWR from 'swr';

// This page is statically exported but fetches data client-side
export default function Dashboard() {
  const { data, error } = useSWR('/api/user', fetcher);
  
  if (error) return <div>Failed to load</div>;
  if (!data) return <div>Loading...</div>;
  
  return <div>Welcome, {data.name}</div>;
}

// Note: API routes won't work in static export unless you proxy them
// Use external API or serverless functions for data
```

## 24.5 Edge Deployment

Deploy to edge networks for global low-latency access.

### Edge Runtime Configuration

Configure routes to run at the edge:

```typescript
// middleware.ts
export const runtime = 'edge'; // Run middleware at edge

export function middleware(request: NextRequest) {
  // Edge-side logic
  const country = request.geo?.country;
  
  if (country === 'US') {
    return NextResponse.rewrite(new URL('/us-version', request.url));
  }
  
  return NextResponse.next();
}

// app/api/edge/route.ts
export const runtime = 'edge'; // Edge API route

export async function GET(request: Request) {
  // This runs at the edge, close to the user
  const { searchParams } = new URL(request.url);
  
  return Response.json({
    message: 'Hello from the edge!',
    timestamp: new Date().toISOString(),
    location: request.headers.get('cf-ray'), // Cloudflare specific
  });
}
```

### Edge-Compatible Database Connections

Use HTTP-based databases for edge compatibility:

```typescript
// lib/edge-db.ts
import { createClient } from '@libsql/client/web';

// Turso (SQLite at edge)
export const turso = createClient({
  url: process.env.TURSO_DATABASE_URL,
  authToken: process.env.TURSO_AUTH_TOKEN,
});

// PlanetScale (serverless MySQL)
import { connect } from '@planetscale/database';

const config = {
  host: process.env.DATABASE_HOST,
  username: process.env.DATABASE_USERNAME,
  password: process.env.DATABASE_PASSWORD,
};

export const conn = connect(config);

// Usage in edge route
export const runtime = 'edge';

export async function GET() {
  const result = await conn.execute('SELECT * FROM users LIMIT 1');
  return Response.json(result.rows[0]);
}
```

## 24.6 CI/CD Pipelines

Automate testing and deployment with continuous integration.

### GitHub Actions Workflow

Complete CI/CD pipeline with GitHub Actions:

```yaml
# .github/workflows/deploy.yml
name: CI/CD Pipeline

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '18'
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Run linter
        run: npm run lint
      
      - name: Run type check
        run: npm run type-check
      
      - name: Run tests
        run: npm test
      
      - name: Run e2e tests
        uses: cypress-io/github-action@v5
        with:
          build: npm run build
          start: npm start

  deploy-preview:
    if: github.event_name == 'pull_request'
    needs: test
    runs-on: ubuntu-latest
    
    steps:
      - uses: actions/checkout@v3
      
      - name: Deploy to Vercel Preview
        uses: vercel/action-deploy@v1
        with:
          vercel-token: ${{ secrets.VERCEL_TOKEN }}
          vercel-org-id: ${{ secrets.ORG_ID }}
          vercel-project-id: ${{ secrets.PROJECT_ID }}
          github-comment: true

  deploy-production:
    if: github.ref == 'refs/heads/main'
    needs: test
    runs-on: ubuntu-latest
    
    steps:
      - uses: actions/checkout@v3
      
      - name: Deploy to Vercel Production
        uses: vercel/action-deploy@v1
        with:
          vercel-token: ${{ secrets.VERCEL_TOKEN }}
          vercel-org-id: ${{ secrets.ORG_ID }}
          vercel-project-id: ${{ secrets.PROJECT_ID }}
          vercel-args: '--prod'
      
      - name: Notify Slack
        uses: 8398a7/action-slack@v3
        with:
          status: ${{ job.status }}
          fields: repo,message,commit,author,action,eventName,ref,workflow
        env:
          SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK_URL }}
```

### Vercel CLI in CI/CD

Deploy using Vercel CLI in custom pipelines:

```yaml
# .github/workflows/vercel-deploy.yml
name: Vercel Deployment

on:
  push:
    branches:
      - main

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2
      
      - name: Install Vercel CLI
        run: npm install --global vercel@latest
      
      - name: Pull Vercel Environment Information
        run: vercel pull --yes --environment=production --token=${{ secrets.VERCEL_TOKEN }}
      
      - name: Build Project Artifacts
        run: vercel build --prod --token=${{ secrets.VERCEL_TOKEN }}
      
      - name: Deploy Project Artifacts to Vercel
        run: vercel deploy --prebuilt --prod --token=${{ secrets.VERCEL_TOKEN }}
```

## 24.7 Environment-Specific Configurations

Manage different settings for development, staging, and production.

### Environment-Based Configuration

Dynamically adjust configuration per environment:

```typescript
// lib/config.ts
type Environment = 'development' | 'staging' | 'production';

const env = (process.env.NODE_ENV as Environment) || 'development';
const vercelEnv = process.env.VERCEL_ENV as Environment | undefined;

// Determine actual environment
const currentEnv = vercelEnv || env;

export const config = {
  env: currentEnv,
  isDevelopment: currentEnv === 'development',
  isProduction: currentEnv === 'production',
  isStaging: currentEnv === 'staging',
  
  // API URLs
  apiUrl: {
    development: 'http://localhost:3000/api',
    staging: 'https://staging.example.com/api',
    production: 'https://example.com/api',
  }[currentEnv],
  
  // Feature flags
  features: {
    analytics: currentEnv === 'production',
    debugMode: currentEnv === 'development',
    maintenance: process.env.MAINTENANCE_MODE === 'true',
  },
  
  // Cache settings
  cache: {
    ttl: currentEnv === 'production' ? 3600 : 0,
    staleWhileRevalidate: currentEnv === 'production' ? 86400 : 0,
  },
  
  // Database pooling
  db: {
    maxConnections: currentEnv === 'production' ? 20 : 5,
    ssl: currentEnv !== 'development',
  },
};
```

### Runtime Environment Validation

Validate environment variables at startup:

```typescript
// env.mjs
import { createEnv } from '@t3-oss/env-nextjs';
import { z } from 'zod';

export const env = createEnv({
  server: {
    DATABASE_URL: z.string().url(),
    NEXTAUTH_SECRET: z.string().min(32),
    STRIPE_SECRET_KEY: z.string().startsWith('sk_'),
    ADMIN_EMAIL: z.string().email().optional(),
  },
  client: {
    NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY: z.string().startsWith('pk_'),
    NEXT_PUBLIC_APP_URL: z.string().url(),
  },
  runtimeEnv: {
    DATABASE_URL: process.env.DATABASE_URL,
    NEXTAUTH_SECRET: process.env.NEXTAUTH_SECRET,
    STRIPE_SECRET_KEY: process.env.STRIPE_SECRET_KEY,
    ADMIN_EMAIL: process.env.ADMIN_EMAIL,
    NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY: process.env.NEXT_PUBLIC_STRIPE_PUBLISHABLE_KEY,
    NEXT_PUBLIC_APP_URL: process.env.NEXT_PUBLIC_APP_URL,
  },
});
```

## 24.8 Monitoring and Analytics

Track application health and performance in production.

### Health Check Endpoints

Implement health checks for load balancers:

```typescript
// app/api/health/route.ts
import { db } from '@/lib/db';
import { redis } from '@/lib/redis';

export const dynamic = 'force-dynamic';

export async function GET() {
  const checks = {
    database: false,
    redis: false,
    timestamp: new Date().toISOString(),
  };

  try {
    await db.$queryRaw`SELECT 1`;
    checks.database = true;
  } catch (error) {
    console.error('Database health check failed:', error);
  }

  try {
    await redis.ping();
    checks.redis = true;
  } catch (error) {
    console.error('Redis health check failed:', error);
  }

  const isHealthy = checks.database && checks.redis;

  return Response.json(
    {
      status: isHealthy ? 'healthy' : 'unhealthy',
      checks,
      uptime: process.uptime(),
    },
    {
      status: isHealthy ? 200 : 503,
      headers: {
        'Cache-Control': 'no-cache, no-store, must-revalidate',
      },
    }
  );
}

// app/api/ready/route.ts
// Kubernetes readiness probe
export async function GET() {
  // Check if app is ready to receive traffic
  return Response.json({ ready: true });
}
```

### Vercel Analytics Integration

Enable Web Vitals tracking:

```typescript
// app/layout.tsx
import { Analytics } from '@vercel/analytics/react';
import { SpeedInsights } from '@vercel/speed-insights/next';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en">
      <body>
        {children}
        <Analytics /> {/* Automatic Web Vitals tracking */}
        <SpeedInsights /> {/* Performance insights */}
      </body>
    </html>
  );
}
```

### Error Tracking and Logging

Integrate error monitoring services:

```typescript
// lib/sentry.ts
import * as Sentry from '@sentry/nextjs';

Sentry.init({
  dsn: process.env.NEXT_PUBLIC_SENTRY_DSN,
  environment: process.env.NODE_ENV,
  tracesSampleRate: process.env.NODE_ENV === 'production' ? 0.1 : 1.0,
  integrations: [
    new Sentry.Integrations.Http({ tracing: true }),
  ],
  beforeSend(event) {
    // Filter out PII
    if (event.request?.headers) {
      delete event.request.headers['cookie'];
      delete event.request.headers['authorization'];
    }
    return event;
  },
});

// app/global-error.tsx
'use client';

import * as Sentry from '@sentry/nextjs';
import Error from 'next/error';

export default function GlobalError({ error }: { error: Error }) {
  useEffect(() => {
    Sentry.captureException(error);
  }, [error]);

  return (
    <html>
      <body>
        <Error statusCode={500} />
      </body>
    </html>
  );
}
```

## Key Takeaways from Chapter 24

1. **Vercel Deployment**: Use `output: 'standalone'` in `next.config.js` for optimized production builds. Configure `vercel.json` for custom headers, CRON jobs, and function duration limits. Use `vercel env` CLI for secure environment variable management across preview and production environments.

2. **Self-Hosting**: Deploy Next.js as a Node.js application using `next start` or PM2 for process management. The standalone output creates a minimal `server.js` with only necessary dependencies, reducing container size. Implement health check endpoints (`/api/health`) for load balancer integration.

3. **Docker Containerization**: Use multi-stage Docker builds (deps → builder → runner) to minimize final image size. Run as non-root user (`USER nextjs`) for security. Set `output: 'standalone'` and copy only `.next/standalone`, `.next/static`, and `public` directories to the runner stage.

4. **Static Exports**: Configure `output: 'export'` for hosting on static CDNs (Cloudflare Pages, Netlify, S3). You must implement `generateStaticParams` for all dynamic routes. Note that API routes, Server Actions, and middleware require actual server deployment—static export is for purely static sites only.

5. **Edge Deployment**: Set `export const runtime = 'edge'` in API routes or middleware to execute code at edge locations close to users. Use HTTP-compatible databases (Turso, PlanetScale, Upstash Redis) since traditional TCP connections aren't supported in Edge Runtime.

6. **CI/CD Pipelines**: Implement GitHub Actions workflows for automated testing (lint, type-check, unit tests, E2E), preview deployments on pull requests, and production deployments on main branch merges. Use Vercel's GitHub integration for automatic preview URLs or manual CLI deployment for custom pipelines.

7. **Environment Management**: Create type-safe environment validation using Zod or `@t3-oss/env-nextjs` to ensure required variables exist at build/start time. Use environment-specific configuration objects to adjust cache TTLs, feature flags, and API endpoints between development, staging, and production.

8. **Monitoring**: Implement `/api/health` endpoints for database and external service connectivity checks. Integrate Vercel Analytics for automatic Core Web Vitals tracking, and Sentry for error tracking with PII filtering. Use structured logging with correlation IDs for distributed tracing in production.

## Coming Up Next

**Chapter 25: Architecture Patterns**

Now that you can deploy your application reliably, it's time to consider how to structure larger applications for maintainability and scale. In Chapter 25, we'll explore Feature-Sliced Design (FSD), layered architecture, monorepo setup with Turborepo, micro-frontends, design systems integration, and scalable project structures. You'll learn how to organize code that grows with your team and application complexity.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. security_best_practices.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. Advanced_architecture_and_patterns/25. architecture_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
